
# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Learning Objectives

At the end of the experiment you will be able to :

-  extract and visualize features for one audio sample.
-  use extracted features to classify them into 30 different classes


**NOTE:** The number of unique classes chosen here is 30. Further, you are encouraged to use different types of classifiers (within SkLearn if you like). Also, try to use sklearn's joblib to save your model (and observe the size of your various ML models). Saving a model is an important step, especially if you wish to export it to another machine/device where you'd like to run your ML algorithm.

In [ ]:
#@title Experiment Walkthrough Video
from IPython.display import HTML

HTML("""<video width="854" height="480" controls>
  <source src="https://cdn.talentsprint.com/aiml/AIML_BATCH_HYD_7/Feb10/mfcc_new.mp4" type="video/mp4">
</video>
""")

## Dataset

### Description

In this experiment we will use TensorFlow’s Speech Commands Datasets which includes 1lakh+ samples in which each sample is a one-second-long utterance of 30 short commands. This dataset has been curated using thousands of people and is opensource under a Creative Commons BY 4.0 license.

Example commands:  'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero'


## Domain Information

When we listen to an audio sample it changes constantly. This means that speech is non-stationary signal. Therefore, normal signal processing techniques cannot be applied to get features from audio. However, if the speech signal is observed using a very small duration window, the speech content in that small duration appears to be  stationary. That brought in the concept of short-time processing of speech. 

MFCC is a technique for short-time processing of speech. 

**Note: While it is a bonus to understand 'how' a speech signal is converted to a 'representation' by MFCC (or other similar speech processing techniques), it is not mandatory to understand the same. It is sufficient to realize that you're generating a unique numeric representation (a set of numbers) of a speech signal and that this unique representation helps you run your machine learning algorithms over the samples.**

## AIML Technique

In this short-time processing technique MFCC, a small duration window (say 25 milli sec) is considered for processing of the audio samples at a time. This small duration is called a frame. Now, for each of the frames, MFCC features are computed which gives a compact representation of the audio samples. A spectrogram is computed for audio samples which gives a heat map of frequencies vs time from the series of spectral vectors generated from audio samples.

This representation is obtained by keeping in mind that humans are much better at grasping small changes in the audio at low frequencies than audio at high frequencies. So mel-scale converts high frequencies to human graspable frequencies. 

In [ ]:
! wget -qq https://cdn.talentsprint.com/aiml/Experiment_related_data/week3/Exp1/AIML_DS_5E1B34A6_NOHASH_0_STD.wav
! mv AIML_DS_5E1B34A6_NOHASH_0_STD.wav?dl=1 AIML_DS_5E1B34A6_NOHASH_0_STD.wav
! wget  https://cdn.talentsprint.com/aiml/Experiment_related_data/week3/Exp1/AIML_DS_AUDIO_STD.zip
! unzip AIML_DS_AUDIO_STD.zip

The above setup cell takes some time to complete the execution, the deep features zip file is around 1.1GB so it takes time to run.

### Importing required packages


In [ ]:
import os

import numpy as np
from scipy import signal
from scipy.io import wavfile
import librosa

# Visualization
import matplotlib.pyplot as plt
import librosa.display

## 1. MFCC features

Let us read a single sample audio file (.wav) from the dataset:

Note: Refer [wavfile.read](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.read.html) from scipy

In [ ]:
filename = 'AIML_DS_5E1B34A6_NOHASH_0_STD.wav'
sample_rate, samples = wavfile.read(str(filename))

In [ ]:
samples = samples.astype('float16')
print(samples)

###  1.1. Amplitude

Speech is a temporal signal, where the amplitude of the signal varies with time.

The amplitude v/s time graph of the audio file we read is:

In [ ]:
fig = plt.figure(figsize=(14, 8))
plt.plot(np.linspace(0, sample_rate/len(samples), sample_rate), samples)
plt.gca().set_title('Raw wave of ' + filename)
plt.gca().set_xlabel('Time')
plt.gca().set_ylabel('Amplitude')
plt.grid("on")

## 1.2 Log Spectrogram

The same speech signal could be interpreted as made up of several frequencies of waves. A visualization of the power, i.e. energy per unit time in each frequency v/s time is called the Spectrogram.

Usually, the log of this energy is considered a better parameter. This is because the power in sound is synonymous with volume (loudness) of the sound, and the human ears are more sensitive to smaller volumes than larger volumes. So it is more convenient to observe the log of the volume rather than the volume itself. The log of sound power is measured in deciBels (dB). (You might be familiar with dB as a unit of sound volume). Hence, we shall consider the Log Spectrogram instead of just the spectrogram.

Let us compute the Log Spectrogram of the audio file we read:

Note: Refer [signal.spectogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html) from scipy

In [ ]:
def log_specgram(audio, sample_rate, window_size=20, step_size=10):
    
    '''
    Divide the signal into equal-length segments. 
    The segments must be short enough that the frequency of the signal does not change appreciably within a segment.
    The segments may or may not overlap.
    Window each segment and compute its spectrum
    '''
    # Number of samples per window/segment
    # window_size represents a number of samples, and a duration
    nperseg = int(round(window_size * sample_rate / 1e3))
    
    # Number of overlapping samples
    # Number of points to overlap between segments
    noverlap = int(round(step_size * sample_rate / 1e3))
    
    # Compute the spectrogram
    freqs, times, spec = signal.spectrogram(audio,
                                            fs=sample_rate,
                                            window='hann',
                                            nperseg=nperseg,
                                            noverlap=noverlap)
    
    # Return log of the spectrogram values, with time axis in columns and frequencies in rows
    return freqs, times, np.log(spec.T)

In [ ]:
# Call the log specgram function
freqs, times, spectrogram = log_specgram(samples, sample_rate)

Let us plot the log spectrogram:

In [ ]:
fig = plt.figure(figsize=(14, 4))
plt.imshow(spectrogram.T, aspect='auto', origin='lower', cmap = plt.cm.RdYlBu,
           extent=[times.min(), times.max(), freqs.min(), freqs.max()])
plt.gca().set_yticks(freqs[::16])
plt.gca().set_xticks(times[9::10])
plt.gca().set_title('Spectrogram of ' + filename)
plt.gca().set_ylabel('Frequency in Hz')
plt.gca().set_xlabel('Time')
plt.colorbar()

As can be seen from the frequency v/s time graph, the command is being uttered between 0.4 seconds and 0.6 seconds.

As can be seen from the spectrogram, the command is composed more of lower frequencies than higher frequencies. The rest of the time when there is no speech, each frequency has an equal contribution to the sound. This is called "White Noise".

Notice that the range of frequencies we are observing in the spectrogram is the linear scale between 0 Hz and 8000 Hz.

### 1.3. Mel Spectrogram

Human ears tend to listen to sounds on the log scale. That means, at lower frequencies we can detect small changes, but at higher frequencies our ears become less sensitive to small changes. For example, the difference between 10 Hz and 20 Hz would sound almost the same to us as that between 1000 Hz and 2000 Hz. To observe this logarithmic change, the frequency scale is modified into the ["mel frequency" scale](https://en.wikipedia.org/wiki/Mel_scale).

Let us compute the Mel Spectrogram using a convenient function in the `librosa` library in Python:

**Note:** Refer to [librosa.feature.melspectogram](https://librosa.org/doc/latest/generated/librosa.feature.melspectrogram.html) here

In [ ]:
# From this tutorial
# https://github.com/librosa/librosa/blob/master/examples/LibROSA%20demo.ipynb

# hop_length is the number of samples between successive frames
S = librosa.feature.melspectrogram(samples, sr=sample_rate, hop_length=int(0.020*sample_rate), n_mels=128)
print(S.shape)

As mentioned before, the log of the spectrogram is a better parameter to observe rather than the spectrogram itself. Let us compute this using another convenient function in the `librosa` library:

In [ ]:
# Convert to log scale (dB). We'll use the peak power (max) as reference.
log_S = librosa.power_to_db(S, ref=np.max)

Let's plot the log Mel spectrogam with the y-axis having frequenies in the mel scale instead of the linear scale: 

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.specshow(log_S, sr=sample_rate, y_axis='mel')
plt.xlabel('Time')
plt.title('Log-power Mel spectrogram ')
plt.colorbar(format='%+02.0f dB')
plt.tight_layout()

Observe that the frequencies in the y-axis are not linear in scale. 

### 1.4 Mel Frequency Cepstral Coefficients (MFCCs)

Next, "Cepstral Coefficients" are important numbers that describe speech information in audio. By computing these Cepstral Coefficients in the mel scale, we shall obtain Mel Frequency Cepstral Coefficients.

For technical details, the procedure to compute MFCCs is:

- Take the Discrete Fourier Transform on every sliding window over the audio with some overlap.
- Apply `n_mels` triangular Mel-scale filters onto the Fourier power spectrum, and apply logarithm to the outputs.
- Apply the Discrete Cosine Transform, and reduce dimensionality to `n_mfcc` dimensions.

Let's use a convenient library function called `librosa.feature.mfcc` to compute MFCCs from the spectrogram:

Note: Refere to [librosa.feature.mfcc](https://librosa.org/doc/latest/generated/librosa.feature.mfcc.html) here

In [ ]:
mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
print(mfcc.shape)

### 1.5 Delta MFCCs

MFCCs as such are quite powerful features, but even better features are their first-order and second-order derivatives.
The mfcc features are then passed to delta which perform 1st and 2nd order differentiation.

Let's use a convenient library function called `librosa.feature.mfcc` to compute the second-order delta MFCCs:

In [ ]:
# Find 1st order delta_mfcc
delta1_mfcc = librosa.feature.delta(mfcc, order=1)

# Find 2nd order delta_mfcc
delta2_mfcc = librosa.feature.delta(mfcc, order=2)

In [ ]:
delta1_mfcc.shape

Let's plot the 1st and 2nd order delta MFCCs:

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(211)
librosa.display.specshow(delta1_mfcc)
plt.ylabel('1st order Delta MFCC coeffs')
plt.xlabel('Time')
plt.title('1st order Delta MFCC')
plt.colorbar()
plt.tight_layout()

plt.subplot(212)
librosa.display.specshow(delta2_mfcc)
plt.ylabel('2nd order Delta MFCC coeffs')
plt.xlabel('Time')
plt.title('2nd order Delta MFCC')
plt.colorbar()
plt.tight_layout()

## 2. Load the Dataset

The dataset is of ~10GB in size and operating directly on it will take a lot of time.
Our team has instead precomputed the features which can be loaded directly and computed on.

Dataset is available to download using the below link: <br>[http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz ](http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz)


## 3. Load MFCC features

NOTE: For the rest of this experiment assume that the term Validation (short name: val) is the same as 'Test' dataset.
Later on in the course you will appreciate the significance of a three way split, i.e. train, val and test. But in this 
 experiment we have two-way Train/Val(same as test) split

**Note:** Refer to [sio.loadmat](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html)

In [ ]:
# Load MFCC Features
import scipy.io as sio
saved_vars = sio.loadmat('AIML_DS_AUDIO_STD/mfcc_feats/tf_speech_mfcc_31st_jan18.mat')
# print(saved_vars.keys())

mfcc_features_train = saved_vars['mfcc_features_train']
mfcc_labels_train = saved_vars['mfcc_labels_train']

mfcc_features_val = saved_vars['mfcc_features_val']
mfcc_labels_val = saved_vars['mfcc_labels_val']

## 4. Load Deep Features

Deep features were trained on the same MFCC Features above using a Deeper Neural network, thus helping us to understand in extracting better features for representing the data. This helps in improving the accuracy

In [ ]:
deep_features_train = np.squeeze(np.load('AIML_DS_AUDIO_STD/deep_feats/train_set.npz'))
deep_labels_train = np.load('AIML_DS_AUDIO_STD/deep_feats/train_labs.npz')
deep_features_val = np.squeeze(np.load('AIML_DS_AUDIO_STD/deep_feats/validation_set.npz'))
deep_labels_val = np.load('AIML_DS_AUDIO_STD/deep_feats/validation_labs.npz')

In [ ]:
deep_labels_train = deep_labels_train.reshape(-1)

In [ ]:
deep_labels_val = deep_labels_val.reshape(-1,1)

## 5. Validation (in this case, synonymous with Test)

Use the ML model for speech classification:

- kNN classifier with MFCC features

- kNN classifier with Deep features


To do that, let's find the validation accuracies using MFCC features and deep features.

We shall use our familiar convenient function to choose an algorithm, train on training features and labels as inputs, and obtain accuracy on given features and labels.

### 5.1. kNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

### 5.1.1. kNN with MFCC features

**Ungraded Exercise 1: Find the validation (in this case, synonymous with Test) accuracy on MFCC features using k=3**

**Note : For quick execution, using only 5000 samples of mfcc_features_train for training  and 1000 samples of mfcc_features_val for validation**

In [ ]:
X_train = mfcc_features_train[:5000, :]
y_train = mfcc_labels_train[:5000]
X_val = mfcc_features_val[:1000, :]
y_val = mfcc_labels_val[:1000]

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# YOUR CODE HERE: To create an object for KNN classifier with 'n_neighbors=3'

# YOUR CODE HERE: To fit the model with the train data

# YOUR CODE HERE: To calculate the accuracy on the validation data

### 5.1.2. kNN with Deep features


**Ungraded Exercise 2: Find the validation (in this case, synonymous with Test) accuracy on Deep features using k=3**


**Note : For quick execution, using only 5000 samples of deep_features_train for training  and 1000 samples of deep_features_val for validation**

In [ ]:
X_train_Deep = deep_features_train[:5000, :]
y_train_Deep = deep_labels_train[:5000]
X_val_Deep = deep_features_val[:1000, :]
y_val_Deep = deep_labels_val[:1000]

In [ ]:
# YOUR CODE HERE: To create an object for KNN classifier with 'n_neighbors=3'

# YOUR CODE HERE: To fit the model with the train data

# YOUR CODE HERE: To calculate the accuracy on the validation data

**Ungraded Exercise 3:** Try various other ML models that you know of to compare the accuracies 

In [ ]:
# YOUR CODE HERE

**Ungraded Exercise 4:** Use the following [link](https://scikit-learn.org/stable/modules/model_persistence.html) to see how to use joblib, and try saving and loading your different ML models using the example provided there. Also try observing the size of the ML models that you saved. Here is another good article on saving and loading models using sklearn [link](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)

In [ ]:
# YOUR CODE HERE